In [1]:
import pandas as pd
import iotbx.pdb
import sys
import numpy as np
import pickle
import math
import matplotlib.pylab as plt
import time
import scipy as sp
import cctbx
import seaborn as sns
from scipy.optimize import fmin, fmin_cg, leastsq, fmin_l_bfgs_b
from scipy.interpolate import interp1d
import scipy.stats 
sns.set_palette("colorblind")
import requests
import re

In [2]:
df = pd.DataFrame()
def read_data(struct):
    rows=[]
    st = iotbx.pdb.input(file_name=struct)
    for atm in st.atoms_with_labels():
        if not atm.element==" H":
            rows.append({ 'b':atm.b,
                          'x': atm.xyz[0],
                          'y': atm.xyz[1],
                          'z': atm.xyz[2],
                          'chain': atm.chain_id,
                          'resname': atm.resname,
                          'Residue number': int(atm.resseq),
                          'combo': atm.icode+atm.resseq+atm.altloc,
                          'altloc': atm.altloc,
                          'Residue_label': atm.icode+atm.resseq+atm.altloc+' '+atm.resname,
                          'Residue_label2': atm.icode+atm.resseq+' '+atm.resname,
                          'atmname': atm.name })
    return pd.DataFrame(rows)

In [3]:


def get_uniprot_entries(pdb_id):
    url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
    response = requests.get(url)
    pdb_data = response.text
    
    uniprot_entries = []
    
    for line in pdb_data.split('\n'):
        if line.startswith('DBREF') and 'UNP' in line:
            match = re.search(r'UNP\s+(\w+)', line)
            if match:
                uniprot_entry = match.group(1)
                chain_id = line[12]
                uniprot_entries.append((chain_id, uniprot_entry))
    
    return uniprot_entries

# Example usage
pdb_id = '7ZTL'  # Replace with your desired PDB ID
uniprot_entries = get_uniprot_entries(pdb_id)
chdict={}
entries=[]
# Print the retrieved chains and UniProt entries
for chain_id, uniprot_entry in uniprot_entries:
    print(f"Chain {chain_id}: UniProt Entry {uniprot_entry}")
    chdict[chain_id]=uniprot_entry
    entries.append(uniprot_entry)
entries= list(set(entries))
entries


Chain A: UniProt Entry O14965
Chain B: UniProt Entry P04198


['O14965', 'P04198']

In [4]:
df=read_data("7ztl.pdb")
#df=read_data("model_20.pdb")

In [5]:
for key in chdict:
    df.loc[df.chain==key,'Entry']=chdict[key]



In [6]:
df

,b,x,y,z,chain,resname,Residue number,combo,altloc,Residue_label,Residue_label2,atmname,Entry
0,80.50,-18.106,30.078,-11.028,A,ARG,126,126,,126 ARG,126 ARG,N,O14965
1,99.15,-18.451,29.272,-12.199,A,ARG,126,126,,126 ARG,126 ARG,CA,O14965
2,90.17,-19.224,28.020,-11.783,A,ARG,126,126,,126 ARG,126 ARG,C,O14965
3,93.00,-20.426,28.084,-11.523,A,ARG,126,126,,126 ARG,126 ARG,O,O14965
4,86.98,-18.527,26.882,-11.739,A,GLN,127,127,,127 GLN,127 GLN,N,O14965
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2584,53.29,-43.561,20.205,21.734,B,HOH,208,208,,208 HOH,208 HOH,O,P04198
2585,66.37,-36.864,17.322,9.218,B,HOH,209,209,,209 HOH,209 HOH,O,P04198
2586,86.32,-57.246,20.373,11.432,B,HOH,210,210,,210 HOH,210 HOH,O,P04198
2587,54.34,-41.093,20.307,21.786,B,HOH,211,211,,211 HOH,211 HOH,O,P04198


In [7]:
# Do not use this!
rows=[]
for entry in entries:
    posres=np.load(entry+'.npz')
    posbind=posres['arr_0']
    for x in posbind.T:
            rows.append({ 'Entry': entry,
                          'Residue number':x[0],
                          'robbind': x[1]
                        })
tf=pd.DataFrame(rows)
tf

,Entry,Residue number,robbind
0,P62799,1.0,0.40
1,P62799,2.0,0.60
2,P62799,3.0,0.45
3,P62799,4.0,0.10
4,P62799,5.0,0.45
...,...,...,...
8125,Q6PI79,131.0,0.30
8126,Q6PI79,132.0,0.25
8127,Q6PI79,133.0,0.30
8128,Q6PI79,134.0,0.10


In [7]:
# Use this!
tfs=[]
for entry in entries:
    _tf=pd.read_excel(entry+'_robbind.xlsx')
    tfs.append(_tf)
tf=pd.concat(tfs)
tf['Residue number']=tf['Position_fasta']
tf

,Unnamed: 0,Position_fasta,Entry,robbind,resname_fasta,Residue number
0,0,1,O14965,0.00,M,1
1,1,2,O14965,0.00,D,2
2,2,3,O14965,0.00,R,3
3,3,4,O14965,0.00,S,4
4,4,5,O14965,0.00,K,5
...,...,...,...,...,...,...
455,455,456,P04198,0.05,K,456
456,456,457,P04198,0.05,K,457
457,457,458,P04198,0.75,I,458
458,458,459,P04198,1.00,E,459


In [8]:
tdf=df[['Entry','chain','Residue number','resname']].drop_duplicates()
cdf=pd.merge(tdf, tf, on=['Entry', 'Residue number'])
cdf=cdf.sort_values(['chain','Residue number'])
cdf.to_excel('cdf_to_check.xlsx')

In [9]:
cdf=pd.read_excel('cdf_checked.xlsx')
#mf = pd.merge(df, tf, on=['Entry', 'Residue number'])
mf = pd.merge(df, cdf, on=['Entry', 'Residue number'])
mf['chain']=mf['chain_x']
mf.to_excel('mf_to_check.xlsx')

In [10]:
mf

,b,x,y,z,chain_x,resname_x,Residue number,combo,altloc,Residue_label,...,atmname,Entry,Unnamed: 0,chain_y,resname_y,Unnamed: 0.1,Position_fasta,robbind,resname_fasta,chain
0,80.50,-18.106,30.078,-11.028,A,ARG,126,126,,126 ARG,...,N,O14965,0,A,ARG,125,126,0.733333,R,A
1,99.15,-18.451,29.272,-12.199,A,ARG,126,126,,126 ARG,...,CA,O14965,0,A,ARG,125,126,0.733333,R,A
2,90.17,-19.224,28.020,-11.783,A,ARG,126,126,,126 ARG,...,C,O14965,0,A,ARG,125,126,0.733333,R,A
3,93.00,-20.426,28.084,-11.523,A,ARG,126,126,,126 ARG,...,O,O14965,0,A,ARG,125,126,0.733333,R,A
4,86.98,-18.527,26.882,-11.739,A,GLN,127,127,,127 GLN,...,N,O14965,1,A,GLN,126,127,1.000000,Q,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2327,53.29,-43.561,20.205,21.734,B,HOH,208,208,,208 HOH,...,O,P04198,290,B,HOH,207,208,0.000000,A,B
2328,66.37,-36.864,17.322,9.218,B,HOH,209,209,,209 HOH,...,O,P04198,291,B,HOH,208,209,0.000000,P,B
2329,86.32,-57.246,20.373,11.432,B,HOH,210,210,,210 HOH,...,O,P04198,292,B,HOH,209,210,0.000000,A,B
2330,54.34,-41.093,20.307,21.786,B,HOH,211,211,,211 HOH,...,O,P04198,293,B,HOH,210,211,0.000000,S,B


In [11]:

rows=[]
st = iotbx.pdb.input(file_name="7ztl.pdb")
for atm in st.atoms_with_labels():
    try:
        if (float(mf[(mf['Residue number']==int(atm.resseq))&(mf['chain']==atm.chain_id)]['robbind'].values[0]))>0:
            atm.set_b(float(mf[(mf['Residue number']==int(atm.resseq))&(mf['chain']==atm.chain_id)]['robbind'].values[0])*100)
            print (float(mf[(mf['Residue number']==int(atm.resseq))&(mf['chain']==atm.chain_id)]['robbind'].values[0])*100)
            #if (mf[(mf['Residue number']==int(atm.resseq))&(mf['chain']==atm.chain_id)]['resname'].values[0])!=atm.resname:
            #    print ('Something is fishy!')
        else:
            atm.set_b(0)
    except:
        atm.set_b(0)


73.33333333333333
73.33333333333333
73.33333333333333
73.33333333333333
100.0
100.0
100.0
100.0
100.0
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
93.33333333333333
100.0
100.0
100.0
100.0
100.0
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
95.0
95.0
95.0
95.0
95.0
95.0
95.0
95.0
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333


31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
96.66666666666667
96.66666666666667
96.66666666666667
96.66666666666667
96.66666666666667
96.66666666666667
96.66666666666667
96.66666666666667
98.33333333333333
98.33333333333333
98.33333333333333
98.33333333333333
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
33.33333333333333
96.66666666666667
96.66666666666667
96.66666666666667
96.66666666666667
96.66666666666667
96.66666666666667
48.33333333333333
48.33333333333333
48.33333333333333
48.33333333333333
48.33333333333333
48.33333333333333
48.33333333333333
48.33333333333333
48.33333333333333
71.66666666666667
71.66666666666667
71.6666666666666

25.0
25.0
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
71.66666666666667
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
25.0
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
36.66666666666666
76.66666666666667
76.66666666666667
76.66666666666667
76.66666666666667
76.66666666666667
76.66666666666667
76.66666666666667
76.66666666666667
76.66666666666667
75.0
75.0
75.0
75.0
75.0
75.0
75.0
58.333333333333336
58.333333333333336
58.333333333333336
58.333333333333336
58.333333333333336
58.333333333333336
58.333333333333336
58.333333333333336
100.0
100.0
100.0
100.0
100.0
100.0
100.0
100.0
80.0
80.0
80.0
80.0
80.0
80.0
80.0
8

23.333333333333332
23.333333333333332
23.333333333333332
23.333333333333332
23.333333333333332
23.333333333333332
23.333333333333332
23.333333333333332
23.333333333333332
53.333333333333336
53.333333333333336
53.333333333333336
53.333333333333336
53.333333333333336
53.333333333333336
31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
31.66666666666667
56.666666666666664
56.666666666666664
56.666666666666664
56.666666666666664
56.666666666666664
56.666666666666664
56.666666666666664
56.666666666666664
56.666666666666664
56.666666666666664
56.666666666666664
56.666666666666664
56.666666666666664
56.666666666666664
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
30.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
75.0
63.33333333333333
63.33333333333

In [13]:
st.write_pdb_file("mapped_7ztl_everything.pdb")

In [37]:
posres=np.load('P62799.npz')
posres['arr_0']

array([[1.00000000e+00, 2.00000000e+00, 3.00000000e+00, 4.00000000e+00,
        5.00000000e+00, 6.00000000e+00, 7.00000000e+00, 8.00000000e+00,
        9.00000000e+00, 1.00000000e+01, 1.10000000e+01, 1.20000000e+01,
        1.30000000e+01, 1.40000000e+01, 1.50000000e+01, 1.60000000e+01,
        1.70000000e+01, 1.80000000e+01, 1.90000000e+01, 2.00000000e+01,
        2.10000000e+01, 2.20000000e+01, 2.30000000e+01, 2.40000000e+01,
        2.50000000e+01, 2.60000000e+01, 2.70000000e+01, 2.80000000e+01,
        2.90000000e+01, 3.00000000e+01, 3.10000000e+01, 3.20000000e+01,
        3.30000000e+01, 3.40000000e+01, 3.50000000e+01, 3.60000000e+01,
        3.70000000e+01, 3.80000000e+01, 3.90000000e+01, 4.00000000e+01,
        4.10000000e+01, 4.20000000e+01, 4.30000000e+01, 4.40000000e+01,
        4.50000000e+01, 4.60000000e+01, 4.70000000e+01, 4.80000000e+01,
        4.90000000e+01, 5.00000000e+01, 5.10000000e+01, 5.20000000e+01,
        5.30000000e+01, 5.40000000e+01, 5.50000000e+01, 5.600000